In [ ]:
import re
import typer
import pandas as pd

from typing import Dict
from pathlib import Path

# <span style="color:purple">Regular Expressions and Their Applications</span>

## Empirical Workshop

### Winter, 2021

<center><img src="../figures/library.png" width="25%" style='border:5px solid #000000'/></center>


* Vast amounts of information are encoded as unstructured data, in the form of text. Fortunately, a lot of it is already stored digitally and is available for computational analysis.

* What tools can we use to perform this analysis?

# <span style="color:purple">The Tool to Use Depends on Text Format and Your Goals</span>

<br>
<br>
<br>

<center><img src="../figures/unstrctured-data-types.png" width="80%" style='border:5px solid #000000'/></center>

<center><img src="../figures/workflow.png" width="80%" style='border:5px solid #000000'/></center>

# <span style="color:purple">Example: Insider Trading Data: SEC Form 4</span>

Form 4 filings are reports submitted to the SEC by investors who buy or sell shares in companies where they are deemed insiders. The SEC defines an insider as any officer, director or more than 10% shareholder of a publicly traded company.

* https://www.sec.gov/files/forms-3-4-5.pdf
* https://www.sec.gov/Archives/edgar/data/1326190/000101297517000759/xslF345X03/edgar.xml
* https://whalewisdomalpha.com/form-4-insider-trading-analysis/

* https://www.sec.gov/Archives/edgar/data/1326190/000101297517000759/

# <span style="color:purple">Regular Expressions: The Swiss Army Knife for Text</span>

<center><img src="../figures/regular_expressions.png" height="100%" style='border:5px solid #000000'/></center>

<center>https://xkcd.com/208/</center>

# <span style="color:purple">What Are Regular Expressions</span>


Regular expressions (called REs, or regexes, or regex patterns) are essentially a tiny, highly specialized programming language. Using this little language, you specify the rules for the set of possible strings that you want to match

* They've been around for decades (https://en.wikipedia.org/wiki/Regular_expression)
* Practically every programming language supports them (e.g, https://docs.python.org/3/howto/regex.html)
* Command line tools such as grep support them (https://en.wikipedia.org/wiki/Grep)

Lot of web tools exist:

* https://regex101.com/
* https://www.debuggex.com/

# <span style="color:purple">Python Example</span>

<br>
<br>
<br>

<center><img src="../figures/create_regex.png" width="50%" style='border:5px solid #000000'/></center>

In [ ]:
# Store the text in a python variable

file = Path("../data/0001012975-17-000759.txt")
text = file.read_text()
typer.secho(text, fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
# Create a pattern string

pat_str = r"^\s*FORMER CONFORMED NAME:(.+?)$"

typer.secho(pat_str, fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
# Compile the pattern string into a pattern object

pattern = re.compile(pat_str, flags=re.DOTALL | re.MULTILINE)

typer.secho(f"{pattern}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
# Match the pattern against text

match = pattern.findall(text)
for m in match:
    typer.secho(f"Match: {m}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
# Split the XML tags using Regex with groups

xml_pat = re.compile(r"<XML>(.+)</XML>", flags=re.DOTALL)
match = xml_pat.findall(text)
xml_text = match[0].strip() 

split_pat = re.compile(r"<(.+)>(.+)<.+>")
match = split_pat.findall(xml_text)
for m in match:
    typer.secho(f"{m}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)
    typer.secho(" " *(len(m[0])+len(m[1]) + 8), bg=typer.colors.WHITE)

# <span style="color:purple">Deeper Dive: Regex Pattern Elements</span>

* Most characters match themselves: <span style="color:blue">A</span> matches "A", <span style="color:blue">9</span> matches "9"

* Sequences of characters match sequences in text: <span style="color:blue">ABC</span> matches "ABC"

* Metacharacters are what allow us to specify abstract patterns: <span style="color:blue">. ^ $ * + ? { } [ ] \ | ( )</span>

* Disjunctions: <span style="color:blue">A|B</span> or <span style="color:blue">[AB]</span>

* Character classes and ranges: <span style="color:blue">[A-Z]</span>, <span style="color:blue">\d</span>, <span style="color:blue">\s</span>, <span style="color:blue">\w</span>, <span style="color:blue">.</span>

* Operators (repetition and optionality): <span style="color:blue">A*</span>, <span style="color:blue">A+</span>, <span style="color:blue">Ab?</span>

* Groups (capture sup-pieces for extraction): <span style="color:blue">A(.+)Z</span>

# <span style="color:purple">Exploring and Checking Data with GREP</span>

<br>
<br>
<br>

<center><img src="../figures/man_grep.png" width="100%"/></center>

# <span style="color:purple">Scaling up to Multiple Documents</span>

<br>
<br>
<br>

<center><img src="../figures/information_extraction.png" width="100%" style='border:5px solid #000000'/></center>

In [ ]:
import re
from typing import Dict

document_fields_header: Dict[str, re.Pattern] = {
    "accession": re.compile(r"ACCESSION NUMBER:(.+?)$", flags=re.DOTALL | re.MULTILINE),
    "sec_document": re.compile(r"<SEC-DOCUMENT>(.+?):", flags=re.DOTALL | re.MULTILINE),
    "sec_header": re.compile(r"<SEC-DOCUMENT>(.+?):", flags=re.DOTALL | re.MULTILINE),
    "acceptance_datetime": re.compile(r"<ACCEPTANCE-DATETIME>(.+?)$", flags=re.DOTALL | re.MULTILINE)
}
    
for key, val in document_fields_header.items():
    typer.secho(f"key: {key}", fg=typer.colors.WHITE, bg=typer.colors.RED)
    typer.secho(f"val: {val}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
def extract_doc_header_info(f: Path) -> Dict[str, str]:
    text = f.read_text()
    row_dict = {"filename": f.name}
    for field, pat in document_fields_header.items():
        row_dict[field] = None
        match = pat.findall(text)
        if match:
            row_dict[field] = match[0].strip()
        else:
            typer.secho(f"WARNING: {f} does not contain {field}", fg=typer.colors.RED)
    return row_dict

typer.secho(f"{extract_doc_header_info}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
from pathlib import Path
import pandas as pd

row_dicts = []
in_dir = Path("../data/2020-sample")
for f in in_dir.glob("*.txt"):
    typer.secho(f"proccessing file: {f.name}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)
    row_dicts.append(extract_doc_header_info(f))

header_df = pd.DataFrame(row_dicts)
header_df = header_df.set_index("filename")

In [ ]:
header_df.head(10)

# <span style="color:purple">Working with XML</span>

https://www.xmlviewer.org/

In [ ]:
from typing import Dict

document_fields: Dict[str, str] = {
    "schemaVersion": "schemaVersion",
    "documentType": "documentType",
    "periodOfReport": "periodOfReport",
    "notSubjectToSection16": "notSubjectToSection16",
    "issuerCik": "issuer/issuerCik",
    "issuerName": "issuer/issuerName",
    "issuerTradingSymbol": "issuer/issuerTradingSymbol"
}
    
for key, val in document_fields.items():
    typer.secho(f"key: {key}", fg=typer.colors.WHITE, bg=typer.colors.RED)
    typer.secho(f"val: {val}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
from typing import Dict
import xml.etree.ElementTree as ET
import re

def extract_doc_xml_info(f: Path) -> Dict[str, str]:
    text = f.read_text()
    row_dict = {"filename": f.name}
    
    # extract the XML portion of the document using a regex
    xml_pat = re.compile(r"<XML>(.+)</XML>", flags=re.DOTALL)
    match = xml_pat.findall(f.read_text())
    xml_text = match[0].strip()   
    root = ET.fromstring(xml_text)

    # iterate through fields and match on path 
    for field, pat in document_fields.items():
        row_dict[field] = None
        match = root.find(pat)
        if match is not None:
            row_dict[field] = match.text.strip()
        else:
            typer.secho(f"WARNING: {f.name} does not contain {field}", bg=typer.colors.BLACK, fg=typer.colors.WHITE)
    
    return row_dict

typer.secho(f"{extract_doc_xml_info}", fg=typer.colors.WHITE, bg=typer.colors.BLACK)

In [ ]:
import typer
import pandas as pd
from pathlib import Path

row_dicts = []
in_dir = Path("../data/2020-sample")
for f in in_dir.glob("*.txt"):
    row_dicts.append(extract_doc_xml_info(f))

xml_df = pd.DataFrame(row_dicts)
xml_df = xml_df.set_index("filename")

In [ ]:
xml_df.head(10)

In [ ]:
# join the results together
df = header_df.join(xml_df)
df.head(10)

# <span style="color:purple">Normalizing Dates with Regex</span>

<br>
<br>
<br>

<center><img src="../figures/MedjoolDates2lb.png" width="800" style='border:5px solid #000000'/></center>

In [ ]:
# Normalizing dates with regex

df["acceptance_datetime"] = df["acceptance_datetime"].str.replace(r'^(\d\d\d\d)(\d\d)(\d\d)(\d*)', r'\1-\2-\3', regex=True)
df.head(10)

# <span style="color:purple">RegEx Resources</span>

* Python: https://docs.python.org/3/howto/regex.html

* R: https://cran.r-project.org/web/packages/stringr/vignettes/regular-expressions.html

* Stata: https://www.stata.com/support/faqs/data-management/regular-expressions/

* If you want to be a master: https://www.amazon.com/dp/0596528124/ref=cm_sw_su_dp

* Interactive web page: https://regex101.com/